In [68]:
import redis
import numpy as np
import pandas as pd
import json

### Atividade Prática – Redis

Em sua conta github, adicione uma pasta com nome redis. Ali, você deve colocar
os fontes e um README.md detalhando como executar o seu programa e um
screenshot demonstrando uma execução feita (eu provavelmente vou estar mais
interessado nos fontes e ver se tudo funcionou)

O Redis tem uma série de estruturas de dados que facilitam nosso dia a dia. Mas
a melhor forma de perceber isso é através do uso com alguns exemplos. Esse
exercício visa permitir a prática de algumas das principais estruturas desse banco
chave valor.

### Redinsgo

In [69]:
r = redis.Redis()

### Criando números para as cartelas ( 1 a 99)

In [70]:
def criar_numeros_cartelas(numero_cartelas=99):
    
    r.delete('numeros')
    
    for i in range(numero_cartelas):
        numero = i+1

        r.sadd('numeros', numero)

### Criando cartelas

In [71]:
def definir_numero_aleatorio(key_cartela):
    
    no_exists = True
    novo_numero = None
    
    while no_exists:
        
        numero = r.srandmember('numeros')
        
        numeros_atuais = r.lrange(key_cartela, 0, -1)
        
        if numero not in numeros_atuais:
            
            no_exists = False
            novo_numero = numero
    
    return novo_numero

In [72]:
def criar_cartelas(quantidade_cartelas=50, quantidade_numeros=15):

    for i in range(quantidade_cartelas):

        cartela = []

        key = f'cartela_{i+1}'
        
        r.delete(key)

        print(f'Cadastrando cartela {key}...')

        for posicao in range(quantidade_numeros):

            numero = definir_numero_aleatorio(key)

            r.rpush(f'cartela_{i+1}', int(numero))

### Criando Usuários

In [73]:
def criar_usuarios(quantidade_usuarios=50):

    for i in range(quantidade_usuarios):

        key = f'user_{i+1}'
        
        r.delete(key)
        
        user = {'name': key, 'bcartela': f'cartela_{i+1}', f'score': 0}

        r.hmset(key, user)

### Fazendo o sorteio

In [74]:
def get_ranking(numero_usuarios=50):
    
    vencedores = []
    
    user = None
    
    for i in range(numero_usuarios):
        
        key_user = f'user_{i+1}'
        
        vencedor = r.hget(key_user, 'score')
        
        vencedores.append((key_user, int(vencedor)))
        
        
    return pd.DataFrame(vencedores, columns=['Usuário', 'Score']).sort_values('Score', ascending=False)

In [75]:
def buscar_vencedores(numero_usuarios=50):
    
    vencedores = []
    
    user = None
    
    for i in range(numero_usuarios):
        
        key_user = f'user_{i+1}'
        
        vencedor = r.hget(key_user, 'vencedor')
        
        if vencedor is not None:
            user = key_user
            break
        
    return user

In [76]:
def atualizar_score(novo_numero_sorteado, numero_usuarios=50):
    
    vencedor = False
    
    for i in range(numero_usuarios):
        
        key_cartela = f'cartela_{i+1}'
        key_user = f'user_{i+1}'
        
        numeros_cartela = r.lrange(key_cartela, 0, -1)
        
        if novo_numero_sorteado in numeros_cartela:
            
            score_atual = int(r.hget(key_user, 'score'))
            
            score_atual = score_atual + 1
            
            r.hset(key_user, 'score', score_atual)
            
            if score_atual == 15:
            
                r.hset(key_user, 'vencedor', 'sim')

In [77]:
def sorteiar():

    numero_nao_sorteado = True

    while numero_nao_sorteado:

        novo_numero_sorteado = r.srandmember('numeros')
        
        numeros_sorteados = r.lrange('jogo', 0, -1)
    
        if novo_numero_sorteado not in numeros_sorteados:
            
            print(f'Numero sorteado = {novo_numero_sorteado}')
            
            r.rpush('jogo', novo_numero_sorteado)
            
            atualizar_score(novo_numero_sorteado)
            
            numero_nao_sorteado = False

In [78]:
def executar_bingo():
    
    print('\nIniciando o bingo...')
    
    nao_tem_vencedor = True
    
    cont=1
    
    while nao_tem_vencedor:
        
        print('\n################')
        print(f'Pedra {cont}')
        sorteiar()
        
        nome_vencedor = buscar_vencedores()
        
        if nome_vencedor is not None:
            nao_tem_vencedor = False
        
        cont = cont + 1
    
    print('\nO bingo terminou.')
    print(f'O vencedor foi o {nome_vencedor}.')

In [79]:
def novo_bingo():
    
    criar_numeros_cartelas()
    criar_cartelas()
    criar_usuarios()
    
    r.delete('jogo')
    
    executar_bingo()
    

In [80]:
novo_bingo()
get_ranking()

Cadastrando cartela cartela_1...
Cadastrando cartela cartela_2...
Cadastrando cartela cartela_3...
Cadastrando cartela cartela_4...
Cadastrando cartela cartela_5...
Cadastrando cartela cartela_6...
Cadastrando cartela cartela_7...
Cadastrando cartela cartela_8...
Cadastrando cartela cartela_9...
Cadastrando cartela cartela_10...
Cadastrando cartela cartela_11...
Cadastrando cartela cartela_12...
Cadastrando cartela cartela_13...
Cadastrando cartela cartela_14...
Cadastrando cartela cartela_15...
Cadastrando cartela cartela_16...
Cadastrando cartela cartela_17...
Cadastrando cartela cartela_18...
Cadastrando cartela cartela_19...
Cadastrando cartela cartela_20...
Cadastrando cartela cartela_21...
Cadastrando cartela cartela_22...
Cadastrando cartela cartela_23...
Cadastrando cartela cartela_24...
Cadastrando cartela cartela_25...
Cadastrando cartela cartela_26...
Cadastrando cartela cartela_27...
Cadastrando cartela cartela_28...
Cadastrando cartela cartela_29...
Cadastrando cartela car

,Usuário,Score
24,user_25,15
10,user_11,14
20,user_21,14
39,user_40,14
13,user_14,14
23,user_24,14
11,user_12,14
25,user_26,14
7,user_8,14
47,user_48,14
